In [1]:
%matplotlib inline

# Street algorithm for mobviz

Add notebook description here. For now, experiments to reshape geojson streets. 

Created on:  2016-10-04  
Last update: 2016-10-24  
Contact: michael.szell@moovel.com, michael.szell@gmail.com (Michael Szell)

In [346]:
# preliminaries
from __future__ import unicode_literals
import sys
import csv
import os
import math
import pprint
import requests
import gzip
from collections import defaultdict
import time
import datetime
import numpy as np
from scipy import stats
import pyprind
import itertools
import logging
from ast import literal_eval as make_tuple
from collections import OrderedDict
from retrying import retry

import json
from shapely.geometry import mapping, shape
import shapely
from scipy import spatial
from haversine import haversine
import overpass
apiop = overpass.API(timeout=600)

# plotting stuff
import matplotlib.pyplot as plt
# import mpld3
# mpld3.enable_notebook() # unfortunately, this is too buggy: https://github.com/mpld3/mpld3/issues/193
# mpld3.disable_notebook()

loadfromfile = False # if True, loads from file, otherwise from overpass API
pathdatain = '/Users/szellmi/Google Drive/MOOVELLAB/00_Projects/mobviz/data/geo/'
pathdataout = '/Users/szellmi/Google Drive/MOOVELLAB/00_Projects/mobviz/data/geo/derived/'
pathdataoutfinal = '/Users/szellmi/Google Drive/MOOVELLAB/00_Projects/mobviz/scripts/mobviz/d3unroll/'
curtime = time.strftime("%Y%m%d_%H%M%S")
logging.basicConfig(filename=pathdataout+'mobviz_streetalgorithm'+curtime+'.log',level=logging.DEBUG)
pp = pprint.PrettyPrinter(indent=4)

# City parameters (generalize later)
cityname = "Berlin"
bbox = "52.33812, 13.0884, 52.675499, 13.76134"
postalcodes = []
with open(pathdatain+"berlin/"+"berlin_postalcodes.txt") as f:
    for line in f:
        postalcodes.append(int(line))

### Functions

In [374]:
def getStreetFeatures(s = "Hannoversche Straße", bb = "52.33812, 13.0884, 52.675499, 13.76134"):
    temp = apiop.Get('node[highway]["name"="'+s+'"]('+bb+'); way[highway]["name"="'+s+'"]('+bb+');')
    # Now throw away all parts where postal code not in city
    js = []
    for f in temp['features']:
        if 'postal_code' in f['properties']:
            if int(f['properties']['postal_code']) in postalcodes:
                js.append(f)
        else:
            js.append(f)
    return {"features":js}

def writeLineStringGEOJSON(filename, streetcoords, path = "/Users/szellmi/Google Drive/MOOVELLAB/00_Projects/mobviz/data/geo/", meta={}):
    with open(path + filename + '.geojson', 'w') as f: #_LineString
        f.write('{"type":"FeatureCollection","features":[{"type":"Feature","properties":'+json.dumps(meta, ensure_ascii=False)+',"geometry":{"type":"LineString","coordinates":')
        f.write(str(streetcoords))
        f.write('}}]}')
        
def writeMultiLineStringGEOJSON(filename, multilinestring, path = "/Users/szellmi/Google Drive/MOOVELLAB/00_Projects/mobviz/data/geo/", meta={}):
    with open(path + filename + '.geojson', 'w') as f:
        f.write('{"type":"MultiLineString","properties":'+json.dumps(meta, ensure_ascii=False)+',"coordinates":')
        f.write(str(multilinestring))
        f.write('}')
        
def getFeaturecollectionFromJS(js, asnparray = True):
    featurecollection = []
    for f in js['features']:    
        s = np.asarray(shape(f['geometry']))
        featurecollection.append(s)
    if not asnparray:
        featurecollection = [i.tolist() for i in featurecollection]
    return featurecollection

def writeFeatureCollectionGEOJSON(filename, featurecollection, path = "/Users/szellmi/Google Drive/MOOVELLAB/00_Projects/mobviz/data/geo/"):
    with open(path + filename + '.geojson', 'w') as f:
        f.write('{"type":"FeatureCollection",')
        temp = json.dumps(featurecollection, ensure_ascii=False)
        f.write(temp[1:])

## Algorithm to stitch together features  from FeatureCollection



The following algorithm assumes that there is an input of a FeatureCollection of different LineStrings, as downloaded via overpass. Each of these LineStrings has a start and an end point. The goal is to provide an output that stitches together these pieces depending on the positions of the start and end points. Occasionally there might be gaps, these have to be filled in.

The input features are assumed to be in this general form: s3 O----O e3  s0 O----O e0  s1 O----O e1  e2 O----O s2  
The algorithm starts to attach to the end point of element 0 the next element. First a=s0, and b=e0. b is updated with the last connection point. This is continued until the distance of the next point to b is larger than to a, in which case all further elements are attached going "left" from s0.  
In this example, the algorithm would output output_connectionpattern = (3,0,1,2), output_forwardbackward = (0,1,1,0). The output output_forwardbackward indicates the direction (s to e, or e to s).

The algorithm is limited to 1-dimensional streets without branches or loops. Complications may arise in these cases.


In [260]:
sid = 'hannoversche'
sname = "Hannoversche Straße"

In [339]:
def getFlattenedStreetFeatures(sname):
    if loadfromfile:
        with open(pathdatain + sname + '.geojson') as f:
            js = json.load(f)
    else:
        js = getStreetFeatures(sname, bbox)
    
    featurecollection = getFeaturecollectionFromJS(js)

    if len(featurecollection) == 1: # nothing to do
        flattenedfeatures = featurecollection[0].tolist()
    else:
        C = list(range(len(featurecollection)))
        output_connectionpattern = [] # outputs the ordered list of features
        output_forwardbackward = [] # outputs for each feature whether LineString direction is forward (1) or backward (0)
        output_gap = [] # outputs whether there is a gap or not, so that one node element can be removed
        startpoints = np.array([featurecollection[i][0] for i in range(len(featurecollection))])
        endpoints = np.array([featurecollection[i][-1] for i in range(len(featurecollection))])
        i = 0 # start with first feature
        output_connectionpattern.extend([i])
        output_forwardbackward.extend([1])
        epsilon = 0.000001 # needed for rounding reasons
        a = startpoints[i]
        stoe = True
        dirinitial = True

        for k in range(len(featurecollection)-1):

            del C[i]
            # Find closest point of s_i/e_i to all other s_j and e_j

            if dirinitial:
                if stoe:
                    b = endpoints[i]
                else:
                    b = startpoints[i]
            else:
                if not stoe:
                    b = endpoints[i]
                else:
                    b = startpoints[i]

            startpoints = np.delete(startpoints, (i), axis=0)
            endpoints = np.delete(endpoints, (i), axis=0)
            SE = np.concatenate((startpoints, endpoints), axis=0)
            # now check the shortest distance of pt to C
            # solve with a k-d tree http://stackoverflow.com/a/32781737
            distancea,indexa = spatial.KDTree(SE).query(a)
            distanceb,indexb = spatial.KDTree(SE).query(b)

            # initially we start going to the right. However, there might be one turn back to the start, which is checked now
            if distancea < distanceb: 
                # a is closer to remaining s_j or e_j, so we now extend to the left 
                # (technically we reverse and still go right)
                output_connectionpattern = output_connectionpattern[::-1]
                output_forwardbackward = output_forwardbackward[::-1]
                output_gap = output_gap[::-1]
                b = a
                dirinitial = False

            # now check the shortest distance of pt to C
            # solve with a k-d tree http://stackoverflow.com/a/32781737
            distance,index = spatial.KDTree(SE).query(b) 
            j = index%(len(featurecollection)-k-1)
            if distance < epsilon:
                output_gap = output_gap + [0]
            else:
                output_gap = output_gap + [1]
            output_connectionpattern = output_connectionpattern + [C[j]]

            if dirinitial:
                if index < (len(featurecollection)-k-1): # closest next point is an s
                    output_forwardbackward = output_forwardbackward + [1]
                    stoe = True
                else:
                    output_forwardbackward = output_forwardbackward + [0]
                    stoe = False
            else:
                if index < (len(featurecollection)-k-1): # closest next point is an s
                    output_forwardbackward = output_forwardbackward + [0]
                    stoe = False
                else:
                    output_forwardbackward = output_forwardbackward + [1]
                    stoe = True

            i = j
    #         print(C[i])


        output_gap = output_gap + [0]
    #     pp.pprint(output_connectionpattern)
    #     pp.pprint(output_forwardbackward)
    #     pp.pprint(output_gap)


        flattenedfeatures = []
        # Now we can reconstruct the line
        for i in range(len(output_connectionpattern)):
            if dirinitial:
                if output_forwardbackward[i]:
                    flattenedfeatures.extend(featurecollection[output_connectionpattern[i]][:].tolist())
                else:
                    flattenedfeatures.extend(reversed(featurecollection[output_connectionpattern[i]][:].tolist()))
            else:
                if not output_forwardbackward[i]:
                    flattenedfeatures.extend(featurecollection[output_connectionpattern[i]][:].tolist())
                else:
                    flattenedfeatures.extend(reversed(featurecollection[output_connectionpattern[i]][:].tolist()))
            if not output_gap[i]:
                del flattenedfeatures[-1]

        # pp.pprint(flattenedfeatures)

        # Export
    #     with open(pathdataout + sname + '.txt', 'w') as f:
    #         f.write(str(flattenedfeatures))
        
    return flattenedfeatures

## Algorithm to hack street into small (10m) parts

In [262]:
seglength = 5 / 1000 # length in km
def hackStreet(flattenedfeatures, seglength = 5/1000):
    hackedstreet = [flattenedfeatures[0]]
    for i,f in enumerate(flattenedfeatures[0:-2]):
        d = haversine((flattenedfeatures[i]), (flattenedfeatures[i+1]), miles=False)
        if d < seglength:
            hackedstreet.append(list(flattenedfeatures[i]))
        else: # hack
            interpoltimes = math.floor(d/seglength)
            for j in range(interpoltimes):
                ip = shapely.geometry.LineString([(flattenedfeatures[i]), (flattenedfeatures[i+1])]).interpolate((j+1)*(1/interpoltimes), normalized=True)
                temp = ip.coords
                hackedstreet.append([temp[0][0], temp[0][1]])
    hackedstreet.append(flattenedfeatures[-1])
    # Export
    with open(pathdataout + streetname + '.txt', 'w') as f:
        f.write(str(hackedstreet))
        
    return hackedstreet

In [219]:
# Write as correct geojson
with open(pathdataoutfinal + 'md' + sid + '.geojson', 'w') as f:
    f.write('{"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"Polygon","coordinates":[')
    f.write(str(hackedstreet))
    f.write(']}}]}')
with open(pathdataoutfinal + sid + '_LineString.geojson', 'w') as f:
    f.write('{"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"LineString","coordinates":')
    f.write(str(hackedstreet))
    f.write('}}]}')


## Complete algorithm to get flattened, hacked street

In [266]:
def getHackedStreet(sname, seglength = 5/1000):
    flattenedfeatures = getFlattenedStreetFeatures(sname)
    return hackStreet(flattenedfeatures, seglength)

## Algorithm to stitch together multiple streets

In [294]:
snames = ['Hannoversche Straße', 'Reichstagufer']
sids = ['hannoversche', 'reichstagufer']
delta = [0,0]
lastpt = [0,0]
streetsstitched = []
for sname, sid in zip(snames, sids):
    hackedstreet = getHackedStreet(sid, 5/1000)
    firstpt = hackedstreet[0]
    delta = [firstpt[0]-lastpt[0], firstpt[1]-lastpt[1]]
    hackedstreet = [['{:.10f}'.format(i[0]-(delta[0])), '{:.10f}'.format(i[1]-(delta[1]))] for i in hackedstreet]
    streetsstitched.extend(hackedstreet)
    lastpt = hackedstreet[-1]
    
    
# remove duplicate coordinates, http://stackoverflow.com/questions/2213923/python-removing-duplicates-from-a-list-of-lists
streetsstitched = list(streetsstitched for streetsstitched,_ in itertools.groupby(streetsstitched))

writeLineStringGEOJSON('multiple1', streetsstitched)

## Download all street features, do not flatten or hack

In [381]:
@retry(wait_exponential_multiplier=1000, wait_exponential_max=600000)
def downloadStreets(sname):
    h = getStreetFeatures(sname)
    #pp.pprint(h)
    #h = getFeaturecollectionFromJS(h, False)
    #meta = {"name": sname}
    #writeMultiLineStringGEOJSON(str(i+1).zfill(5), h, "/Users/szellmi/Google Drive/MOOVELLAB/00_Projects/mobviz/data/geo/streetsraw/", meta)
    writeFeatureCollectionGEOJSON(str(i+1+lagi).zfill(5), h, "/Users/szellmi/Google Drive/MOOVELLAB/00_Projects/mobviz/data/geo/streetsraw/")

with open(pathdatain + 'berlin_streetnames.txt') as f:
    snames = []
    for line in f:
        snames.append(line.rstrip('\n'))
bar = pyprind.ProgBar(len(snames), bar_char='█', update_interval=1)

lagi = 470
for i, sname in enumerate(snames[0+lagi:]):
    bar.update(item_id = sname)
    downloadStreets(sname)
    

0%                          100%
[                              ]

KeyboardInterrupt: 

## Download all street features, flatten (and hack)

In [348]:
@retry(wait_exponential_multiplier=1000, wait_exponential_max=600000)
def downloadStreetsFlatten(sname):
    h = getFlattenedStreetFeatures(sname) # if hacked too, use: getHackedStreet
    meta = {"name": sname}
    writeLineStringGEOJSON(str(i+1).zfill(5), h, "/Users/szellmi/Google Drive/MOOVELLAB/00_Projects/mobviz/data/geo/streetsflattened/", meta)

with open(pathdatain + 'berlin_streetnames.txt') as f:
    snames = []
    for line in f:
        snames.append(line.rstrip('\n'))
bar = pyprind.ProgBar(len(snames), bar_char='█', update_interval=1)

for i, sname in enumerate(snames):
    bar.update(item_id = sname)
    downloadStreetsFlatten(sname)
    
    

0%                          100%
[                              ] | ETA: 16:48:56 | Item ID: Albert-Einstein-Ring

KeyboardInterrupt: 